In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

# Chapter 6. 다양한 데이터 타입 다루기
## 6.1 API는 어디서 찾을까?
+ DataFrame 메서드
    + DataFrameStatFunctions : 다양한 통계적 함수를 제공
    + DataFrameNaFunctions : null 데이터를 다루는데 필요한 함수를 제공
+ Column 메서드
    + alias, contains과 같은 컬럼과 관련된 여러가지 메서드를 제공
    + org.apache.spark.sql.function 패키지는 데이터 타입과 관련된 다양한 함수를 제공

In [2]:
""" DataFrame 생성 """
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("../BookSamples/data/retail-data/by-day/2010-12-01.csv")
df.printSchema()
df.createOrReplaceTempView("dfTable")
df.show(5)

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   8

## 6.2 스파크 데이터 타입으로 변환하기
+ lit 함수 : 다른 언어의 데이터 타입을 스파크 데이터 타입에 맞게 변환
+ SQL에서는 스파크 데이터 타입으로 변환할 필요가 없으므로 값을 직접 입력해 사용

In [3]:
""" 스파크 데이터 타입으로 변환 """
from pyspark.sql.functions import lit

df.select(lit(5), lit("five"), lit(5.0))

DataFrame[5: int, five: string, 5.0: double]

## 6.3 불리언 데이터 타입 다루기

In [4]:
""" 조건절 입력 """
from pyspark.sql.functions import col

# 컬럼 함수 사용
df.where(col("InvoiceNO") != 536365) \
    .select("InvoiceNO", "Description") \
    .show(5, False)

# 가장 명확한 방법은 문자열 표현식에 조건절을 명시하는 것
df.where("InvoiceNO = 536365") \
    .show(5, False) # truncate = False

df.where("InvoiceNO <> 536365") \
    .show(5, True)

+---------+-----------------------------+
|InvoiceNO|Description                  |
+---------+-----------------------------+
|536366   |HAND WARMER UNION JACK       |
|536366   |HAND WARMER RED POLKA DOT    |
|536367   |ASSORTED COLOUR BIRD ORNAMENT|
|536367   |POPPY'S PLAYHOUSE BEDROOM    |
|536367   |POPPY'S PLAYHOUSE KITCHEN    |
+---------+-----------------------------+
only showing top 5 rows

+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |2010-12-01 08:26:00|2.55     |17850.0   |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |2010-12-01 08:26:00|3.39     |17850.0  

In [5]:
""" an, or 사용한 불리언 표현식 """

from pyspark.sql.functions import instr # contains

priceFilter = col("UnitPrice") > 600
descripFilter = instr(df.Description, "POSTAGE") >= 1 # Locate the position of the first occurrence of substr column in the given string.
                                                      # Returns null if either of the arguments are null.
df.where(df.StockCode.isin("DOT")).where(priceFilter | descripFilter).show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



In [6]:
""" instr 함수 """

df.withColumn("added", instr(df.Description, "POSTAGE")).where("added > 1").show() # 8번째 글자에 'POSTAGE'가 시작됨

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|added|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|    8|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|    8|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----+



In [7]:
""" 불리언 컬럼 """
from pyspark.sql.functions import instr

DOTCodeFilter = col("StockCode") == "DOT" # column type
priceFilter = col("UnitPrice") > 600
descripFilter = instr(col("Description"), "POSTAGE") > 1 # 'POSTAGE' 글자를 포함하고 있는 경우
df.withColumn("isExpensive", DOTCodeFilter & (priceFilter | descripFilter)) \
    .where("isExpensive") \
    .select("unitPrice", "isExpensive").show(5)

+---------+-----------+
|unitPrice|isExpensive|
+---------+-----------+
|   569.77|       true|
|   607.49|       true|
+---------+-----------+



In [8]:
col("StockCode") == "DOT"

Column<b'(StockCode = DOT)'>

In [9]:
""" SQL문 실행 """
from pyspark.sql.functions import expr, col # 파이썬은 not이 존재하지 않는다. 

df.withColumn("isExpensive", expr("NOT UnitPrice <= 250")) \
    .where("isExpensive") \
    .select("Description", "UnitPrice").explain()
# SQL을 사용해도 성능은 차이나지 않는다. -> spark 고유의 데이터 타입 활용

== Physical Plan ==
*(1) Project [Description#18, UnitPrice#21]
+- *(1) Filter (isnotnull(UnitPrice#21) AND (UnitPrice#21 > 250.0))
   +- FileScan csv [Description#18,UnitPrice#21] Batched: false, DataFilters: [isnotnull(UnitPrice#21), (UnitPrice#21 > 250.0)], Format: CSV, Location: InMemoryFileIndex[file:/home/jovyan/work/BookSamples/data/retail-data/by-day/2010-12-01.csv], PartitionFilters: [], PushedFilters: [IsNotNull(UnitPrice), GreaterThan(UnitPrice,250.0)], ReadSchema: struct<Description:string,UnitPrice:double>




## 6.4 수치형 데이터 타입 다루기

+ 아래 예제의 두 컬럼 모두 수치형이므로 곱셈 연산이 가능함. 필요한 경우 덧셈이나 뺄셈도 가능
+ SQL을 사용해 동일하게 처리할 수도 있음

In [10]:
""" 지수만큼 제곱하는 pow 함수 """
from pyspark.sql.functions import expr, pow

fabricateQuantity = pow(col("Quantity") * col("UnitPrice"), 2) + 5
df.select(expr("CustomerID"), fabricateQuantity.alias("realQuantity")).show(5)

+----------+------------------+
|CustomerID|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
|   17850.0|             489.0|
|   17850.0|          418.7156|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 5 rows



In [11]:
""" SQL문으로 위와 동일한 결과 실행 """
df.selectExpr(
    "CustomerId",
    "(POWER(Quantity * UnitPrice, 2.0) + 5) as realQuantity"
).show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



+ round: 소수점 값이 정확히 중간값 이상이라면 반올림
+ bround: 반내림

In [12]:
""" 반올림하는 round 함수 """
from pyspark.sql.functions import lit, round, bround # round(반올림), bround(반내림)

df.select(round(lit("2.5"), 0), bround(lit("2.5"), 0)).show(2) 
df.select(round(lit("2.5"), 1), bround(lit("2.5"), 1)).show(2) # 1차원
df.select(round(lit("2.55"), 1), bround(lit("2.55"), 1)).show(2) #??

+-------------+--------------+
|round(2.5, 0)|bround(2.5, 0)|
+-------------+--------------+
|          3.0|           2.0|
|          3.0|           2.0|
+-------------+--------------+
only showing top 2 rows

+-------------+--------------+
|round(2.5, 1)|bround(2.5, 1)|
+-------------+--------------+
|          2.5|           2.5|
|          2.5|           2.5|
+-------------+--------------+
only showing top 2 rows

+--------------+---------------+
|round(2.55, 1)|bround(2.55, 1)|
+--------------+---------------+
|           2.6|            2.6|
|           2.6|            2.6|
+--------------+---------------+
only showing top 2 rows



+ class pyspark.sql.DataFrameStatFunctions : Functionality for statistic functions with DataFrame.

In [13]:
""" 피어슨 상관계수를 계산 """
from pyspark.sql.functions import corr

df.stat.corr("Quantity", "UnitPrice")
df.corr("Quantity", "UnitPrice") # dataframe에서 바로 접근도 가능함

-0.04112314436835551

+ 요약 통계는 describe메서드를 사용해 얻을 수 있음. 
+ describe메서드는 관련 컬럼에 대한 집계(count), 평균(mean), 표준편차(stddev), 최솟값(min), 최댓값(max)를 계산

In [14]:
""" 요약 통계를 계산 """
df.describe().show()
df.describe("InvoiceNo").show() # 컬럼을 입력

+-------+-----------------+------------------+--------------------+------------------+-------------------+------------------+------------------+--------------+
|summary|        InvoiceNo|         StockCode|         Description|          Quantity|        InvoiceDate|         UnitPrice|        CustomerID|       Country|
+-------+-----------------+------------------+--------------------+------------------+-------------------+------------------+------------------+--------------+
|  count|             3108|              3108|                3098|              3108|               3108|              3108|              1968|          3108|
|   mean| 536516.684944841|27834.304044117645|                null| 8.627413127413128|               null| 4.151946589446603|15661.388719512195|          null|
| stddev|72.89447869788873|17407.897548583845|                null|26.371821677029203|               null|15.638659854603892|1854.4496996893627|          null|
|    min|           536365|             

+ 정확한 수치가 필요하다면 함수를 임포트하고 해당 컬럼에 적용하는 방식으로 직접 집계를 수행할 수 있음

In [15]:
from pyspark.sql.functions import count, mean, stddev_pop, min, max

df.select(stddev_pop(col("UnitPrice"))).show()
df.select(stddev_pop("UnitPrice")).show()

+---------------------+
|stddev_pop(UnitPrice)|
+---------------------+
|   15.636143780280698|
+---------------------+

+---------------------+
|stddev_pop(UnitPrice)|
+---------------------+
|   15.636143780280698|
+---------------------+



+ StaFunctions 패키지는 다양한 통계 함수를 제공
+ stat 속성을 사용해 접근할 수 있으며 다양한 통곗값을 계산할 때 사용하는 DataFrame 메서드

+ approxQuantile:
    + The result of this algorithm has the following deterministic bound: If the DataFrame has N elements and if we request the quantile at probability p up to error err, then the algorithm will return a sample x from the DataFrame so that the exact rank of x is close to (p * N). 
    + More precisely, `floor((p - err) * N) <= rank(x) <= ceil((p + err) * N)`

In [16]:
""" 백분위수를 구하는 approxQuantile """
olName = "UnitPrice"
quantileProbs = [0.5]
relError = 0.05

df.stat.approxQuantile("UnitPrice", quantileProbs, relError)
# :relError: The relative target precision to achieve
#   (>= 0). If set to zero, the exact quantiles are computed, which
#   could be very expensive. Note that values greater than 1 are
#   accepted but give the same result as 1.

[2.51]

In [17]:
df.where("UnitPrice = 2.51").show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536544|    21773|DECORATIVE ROSE B...|       1|2010-12-01 14:32:00|     2.51|      null|United Kingdom|
|   536544|    21774|DECORATIVE CATS B...|       2|2010-12-01 14:32:00|     2.51|      null|United Kingdom|
|   536544|    21791|VINTAGE HEADS AND...|       2|2010-12-01 14:32:00|     2.51|      null|United Kingdom|
|   536544|    21809|CHRISTMAS HANGING...|       1|2010-12-01 14:32:00|     2.51|      null|United Kingdom|
|   536544|    21810|CHRISTMAS HANGING...|       3|2010-12-01 14:32:00|     2.51|      null|United Kingdom|
|   536544|    21811|CHRISTMAS HANGING...|       1|2010-12-01 14:32:00|     2.51|      null|United Kingdom|
|   536544|    21889|WOODEN 

+ crosstab : Computes a pair-wise frequency table of the given columns. Also known as a contingency table. 
    + The first column of each row will be the distinct values of col1 and the column names will be the distinct values of col2. 
    + The name of the first column will be `$col1_$col2`. 
    + Pairs that have no occurrences will have zero as their counts. 
    + DataFrame.crosstab() and DataFrameStatFunctions.crosstab() are aliases.


In [18]:
""" 교차표를 생성하는 crosstab """
df.select("StockCode").distinct().show() # rows name
df.select("Quantity").distinct().show()  # columns name
df.stat.crosstab("StockCode", "Quantity").show() # pivot

+---------+
|StockCode|
+---------+
|    22728|
|    21889|
|   90210B|
|    21259|
|    21894|
|    21452|
|    22121|
|    90022|
|    21249|
|    21711|
|    22130|
|    22314|
|    21671|
|    22629|
|    82486|
|    22438|
|    22529|
|    10133|
|    21967|
|    85064|
+---------+
only showing top 20 rows

+--------+
|Quantity|
+--------+
|      34|
|      -1|
|      28|
|      27|
|     384|
|     -10|
|     192|
|      12|
|     128|
|      22|
|      47|
|       1|
|      13|
|       6|
|      16|
|       3|
|      20|
|      40|
|      -7|
|     432|
+--------+
only showing top 20 rows

+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|StockCode_Quantity| -1|-10|-12| -2|-24| -3| -4| -5| -6| -7|  1| 10|100| 11| 12|120|128| 13| 14|144| 15| 16| 17

+ monotonically_increasing_id:
    + 모든 로우에 고유 ID값을 추가함. 모든 로우에 0부터 시작하는 고윳값을 생성
    + The generated ID is guaranteed to be monotonically increasing and unique, but not consecutive. 
    + The current implementation puts the partition ID in the upper 31 bits, and the record number within each partition in the lower 33 bits

In [19]:
""" 고유 아이디를 생성하는 monotonically_increasing_id """
from pyspark.sql.functions import monotonically_increasing_id

df.select(monotonically_increasing_id()).show(10)

+-----------------------------+
|monotonically_increasing_id()|
+-----------------------------+
|                            0|
|                            1|
|                            2|
|                            3|
|                            4|
|                            5|
|                            6|
|                            7|
|                            8|
|                            9|
+-----------------------------+
only showing top 10 rows



In [20]:
df2 = df.repartition(2)
df2.select(monotonically_increasing_id().alias('id')) \
    .show(10)
df2.select(monotonically_increasing_id().alias('id')) \
    .orderBy(col('id').desc()) \
    .show(10)

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+
only showing top 10 rows

+----------+
|        id|
+----------+
|8589936145|
|8589936144|
|8589936143|
|8589936142|
|8589936141|
|8589936140|
|8589936139|
|8589936138|
|8589936137|
|8589936136|
+----------+
only showing top 10 rows



## 6.5 문자열 데이터 타입 다루기

In [21]:
""" 공백으로 나뉘는 모든 단어의 첫 글자를 대문자로 변경, initcap """ 
from pyspark.sql.functions import initcap

df.select(initcap(col("Description"))).show(2, False)

+----------------------------------+
|initcap(Description)              |
+----------------------------------+
|White Hanging Heart T-light Holder|
|White Metal Lantern               |
+----------------------------------+
only showing top 2 rows



In [22]:
""" 소문자/대문자 변경, lower/upper """
from pyspark.sql.functions import lower, upper

df.select(col("Description"), lower(col("Description")), upper(col("Description"))).show(2)

+--------------------+--------------------+--------------------+
|         Description|  lower(Description)|  upper(Description)|
+--------------------+--------------------+--------------------+
|WHITE HANGING HEA...|white hanging hea...|WHITE HANGING HEA...|
| WHITE METAL LANTERN| white metal lantern| WHITE METAL LANTERN|
+--------------------+--------------------+--------------------+
only showing top 2 rows



In [23]:
""" 문자열 주변의 공백을 제거, lpad/ltrim/rpad/rtrim/trim """
from pyspark.sql.functions import lit, ltrim, rtrim, rpad, lpad, trim

df.select(
    ltrim(lit("   HELLO   ")).alias("ltrim"),
    rtrim(lit("   HELLO   ")).alias("rtrim"),
    trim(lit("   HELLO   ")).alias("trim"),
    lpad(lit("HELLO"), 3, " ").alias("lp"),
    rpad(lit("HELLO"), 10, " ").alias("rp")
).show(2)

+--------+--------+-----+---+----------+
|   ltrim|   rtrim| trim| lp|        rp|
+--------+--------+-----+---+----------+
|HELLO   |   HELLO|HELLO|HEL|HELLO     |
|HELLO   |   HELLO|HELLO|HEL|HELLO     |
+--------+--------+-----+---+----------+
only showing top 2 rows



### 6.5.1 정규 표현식
+ 존재 여부를 확인하거나 일치하는 모든 문자열을 치환
+ 정규 표현식을 위해 `regexp_extract` 함수와 `regexp_replace` 함수를 제공

In [24]:
""" 단어 치환, regexp_extract """
from pyspark.sql.functions import regexp_replace

regex_string = "BLACK|WHITE|RED|GRENN|BLUE"
df.select(
    regexp_replace(col("Description"), regex_string, "COLOR").alias("color_clean"),
    col("Description")).show(2)

+--------------------+--------------------+
|         color_clean|         Description|
+--------------------+--------------------+
|COLOR HANGING HEA...|WHITE HANGING HEA...|
| COLOR METAL LANTERN| WHITE METAL LANTERN|
+--------------------+--------------------+
only showing top 2 rows



In [25]:
""" 문자 단위 치환, translate """
from pyspark.sql.functions import translate

df.select(translate(col("Description"), "LEET", "12"), # 매칭되지 않아도 실행되니 주의 (L=1, E=2, T='' 으로 치환됨)
          col("Description")).show(2)

+--------------------------------+--------------------+
|translate(Description, LEET, 12)|         Description|
+--------------------------------+--------------------+
|            WHI2 HANGING H2AR...|WHITE HANGING HEA...|
|                WHI2 M2A1 1AN2RN| WHITE METAL LANTERN|
+--------------------------------+--------------------+
only showing top 2 rows



+ regexp_extract(str, pattern, idx):
    + Extract a specific group matched by a Java regex, from the specified string column. If the regex did not match, or the specified group did not match, an empty string is returned.

In [26]:
""" 단어 추출, regexp_extract """
from pyspark.sql.functions import regexp_extract

# 처음 나타난 색상 이름을 추출
extract_str = "(BLACK|WHITE|RED|GRENN|BLUE)"
df.select(
    regexp_extract(col("Description"), extract_str, 1).alias("color_clean"),
    col("Description")).show(2)

+-----------+--------------------+
|color_clean|         Description|
+-----------+--------------------+
|      WHITE|WHITE HANGING HEA...|
|      WHITE| WHITE METAL LANTERN|
+-----------+--------------------+
only showing top 2 rows



In [27]:
""" 단어 존재유무, contain """ # 파이썬과 SQL은 instr 함수를 사용
from pyspark.sql.functions import instr

containBlack = instr(col("Description"), "BLACK") >= 1
containWhite = instr(col("Description"), "WHITE") >= 1
df.withColumn("hasSimpleColor", containBlack | containWhite) \
    .where("hasSimpleColor") \
    .select("Description").show(3, False) 

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
+----------------------------------+
only showing top 3 rows



+ 동적으로 인수의 개수가 변화는 상황을 스파크는 어떻게 처리하나?
    + 값 목록을 인수로 변환해 함수에 전달할 때는 varargs라 불리는 스칼라 고유 기능을 활용 -> 임의 길이의 배열을 효율적으로 다룰 수 있음
    + 파이썬은 인수의 개수가 동적으로 변화는 상황을 아주 쉽게 해결할 수 있음

+ 문자열의 위치(위치는 1부터 시작)를 정수로 반환하는 locate 함수를 사용, 그런다음 위치 정보를 불리언 타입으로 변환

In [28]:
""" 인수의 개수가 동적으로 변하는 상황과 문자열의 위치를 찾는 locate 함수 """
from pyspark.sql.functions import expr, locate 

simple_colors = ["black", "white", "red", "green", "blue"]
def color_locator(column, color_string):
    return locate(color_string.upper(), column).cast("boolean").alias("is_" + color_string) # color_strings 단어가 시작되는 문자기준(단어기준 X) 위치
selected_cols = [color_locator(df.Description, c) for c in simple_colors] # locate 함수를 하나씩 list에 저장
selected_cols.append(expr("*")) # column 타입이여야 함 

df.select(*selected_cols).show(3)

df.select(*selected_cols).where(expr("is_white OR is_red")) \
    .select(col("Description")).show(3, False)

+--------+--------+------+--------+-------+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|is_black|is_white|is_red|is_green|is_blue|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+--------+--------+------+--------+-------+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   false|    true| false|   false|  false|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   false|    true| false|   false|  false|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   false|   false| false|   false|  false|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
+--------+--------+------+--------+-------+---------+---------+--------------------+--------+-------

In [29]:
selected_cols = [color_locator(df.Description, c) for c in simple_colors] # locate 함수를 하나씩 list에 저장
selected_cols

[Column<b'CAST(locate(BLACK, Description, 1) AS BOOLEAN) AS `is_black`'>,
 Column<b'CAST(locate(WHITE, Description, 1) AS BOOLEAN) AS `is_white`'>,
 Column<b'CAST(locate(RED, Description, 1) AS BOOLEAN) AS `is_red`'>,
 Column<b'CAST(locate(GREEN, Description, 1) AS BOOLEAN) AS `is_green`'>,
 Column<b'CAST(locate(BLUE, Description, 1) AS BOOLEAN) AS `is_blue`'>]

## 6.6 날짜와 타임스팸프 데이터 타입 다루기
+ 스파크는 2가지 시간 정보만 다룸
    + 날짜 정보만 가지는 date
    + 날짜와 시간 정보를 모두 가지는 timestamp
+ 시간대 설정이 필요하다면 스파크 SQL 설정의 spark.conf.sessionLocalTimeZone 속성으로 가능
    + [자바 TimeZone 포맷을 따라야 함](https://docs.oracle.com/javase/7/docs/api/java/util/TimeZone.html)
+ TimestampType 클래스는 초 단위 정밀도만 지원   
    + 초 단위 이상 정밀도 요구 시 long 데이터 타입으로 데이터를 변환해 처리하는 우회 정책이 필요

In [30]:
""" 오늘 날짜와 현재의 타임스탬프값 구하기 """
from pyspark.sql.functions import current_date, current_timestamp

dateDF = spark.range(10) \
    .withColumn("today", current_date())\
    .withColumn("now", current_timestamp())
dateDF.createOrReplaceTempView("dataTable")
dateDF.printSchema()

dateDF.show(3, False)

root
 |-- id: long (nullable = false)
 |-- today: date (nullable = false)
 |-- now: timestamp (nullable = false)

+---+----------+--------------------------+
|id |today     |now                       |
+---+----------+--------------------------+
|0  |2020-11-14|2020-11-14 23:55:07.602953|
|1  |2020-11-14|2020-11-14 23:55:07.602953|
|2  |2020-11-14|2020-11-14 23:55:07.602953|
+---+----------+--------------------------+
only showing top 3 rows



In [31]:
""" 오늘을 기준으로 날짜를 더하거나 빼기 """
from pyspark.sql.functions import date_sub, date_add

dateDF.select(
    date_sub(col("today"), 5),
    date_add(col("today"), 5)
).show(1)

+------------------+------------------+
|date_sub(today, 5)|date_add(today, 5)|
+------------------+------------------+
|        2020-11-09|        2020-11-19|
+------------------+------------------+
only showing top 1 row



In [32]:
""" 두 날짜 사이의 일/개월 수를 파악 """
from pyspark.sql.functions import datediff, months_between, to_date

dateDF.withColumn("week_ago", date_sub(col("today"), 7))\
    .select(datediff(col("week_ago"), col("today"))).show(1) # 현재 날짜에서 7일 제외 후 datediff 결과 확인
dateDF.select(
    to_date(lit("2016-01-01")).alias("start"), 
    to_date(lit("2017-05-22")).alias("end"))\
    .select(months_between(col("start"), col("end"))).show(1) # 개월 수 차이 파악

+-------------------------+
|datediff(week_ago, today)|
+-------------------------+
|                       -7|
+-------------------------+
only showing top 1 row

+--------------------------------+
|months_between(start, end, true)|
+--------------------------------+
|                    -16.67741935|
+--------------------------------+
only showing top 1 row



+ to_date: 문자열을 날짜로 변경할 수 있으며, 필요에 따라 날짜 포맷도 함께 지정하 수 있음.
    + 함수의 날짜 포맷은 반드시 자바의 SimpleDateFormat클래스가 지원하는 포맷을 사용해야 함

In [33]:
""" 문자열을 날짜로 변환 """ # 자바의 simpleDateFormat 클래스가 지원하는 포맷 사용 필요
from pyspark.sql.functions import to_date, lit

df_date = spark.range(5).withColumn("date", lit("2017-01-01"))\
    .select(to_date(col("date")))
df_date.printSchema()
df_date.show(1)


root
 |-- to_date(`date`): date (nullable = true)

+---------------+
|to_date(`date`)|
+---------------+
|     2017-01-01|
+---------------+
only showing top 1 row



+ 날짜를 파싱할 수 없다면 에러 대신 null값을 반환함

In [34]:
""" 파싱오류로 날짜가 null로 반환되는 사례 """
dateDF.select(to_date(lit("2016-20-12")), to_date(lit("2017-12-11"))).show(1) # 월과 일의 순서가 바뀜

+---------------------+---------------------+
|to_date('2016-20-12')|to_date('2017-12-11')|
+---------------------+---------------------+
|                 null|           2017-12-11|
+---------------------+---------------------+
only showing top 1 row



+ 날짜 형식을 지키지 않은 데이터가 들어온다면 디버깅하기 매우 어려움
    + 자바의 SimpleDateFormat 표준에 맞춰 날짜 포맷 지정
    + to_date 함수는 필요에 따라 날짜 포맷을 지정할 수 있지만, to_timestamp함수는 반드시 날짜 포맷을 지정해야 함
    

In [35]:
""" SimpleDateFormat 표준을 활용하여 날짜 포멧을 지정 """
from pyspark.sql.functions import to_date
dateFormat = "yyyy-dd-MM" # 소문자 mm 주의
cleanDateDF = spark.range(1).select( # 1개 Row를 생성
    to_date(lit("2017-12-11"), dateFormat).alias("date"),
    to_date(lit("2017-20-12"), dateFormat).alias("date2"))
cleanDateDF.createOrReplaceTempView("dateTable2")

※ SimpleDateFormat : https://bvc12.tistory.com/168

In [36]:
spark.sql("""SELECT * from dateTable2""").show()

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+



In [37]:
""" 항상 날짜 포맷을 지정해야 하는 to_timestamp 함수 """
from pyspark.sql.functions import to_timestamp

cleanDateDF.select(to_timestamp(col("date"), dateFormat)).show()

+----------------------------------+
|to_timestamp(`date`, 'yyyy-dd-MM')|
+----------------------------------+
|               2017-11-12 00:00:00|
+----------------------------------+



In [38]:
""" 날짜 비교 """
cleanDateDF.filter(col("date2") > lit("2017-12-12")).show()

+----------+----------+
|      date|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+



## 6.7 null 값 다루기
+ null 값을 사용하는 것 보다 명시적으로 사용하는 것이 항상 좋음
+ null 값을 허용하지 않는 컬럼을 선언해도 강제성은 없음
+ nullable 속성은 스파크 SQL 옵티마이저가 해당 컬럼을 제어하는 동작을 단순하게 돕는 역할
+ null 값을 다루는 방법은 두 가지 
    + 명시적으로 null을 제거
    + 전역 또느 컬럼 단위로 null 값을 특정 값으로 채움

### 6.7.1 coalesce

In [39]:
from pyspark.sql.functions import coalesce # 인수로 지정한 여러 컬럼 중 null이 아닌 첫번 째 값 반환

df.select(coalesce(col("Description"), col("CustomerId"))).show()

+---------------------------------+
|coalesce(Description, CustomerId)|
+---------------------------------+
|             WHITE HANGING HEA...|
|              WHITE METAL LANTERN|
|             CREAM CUPID HEART...|
|             KNITTED UNION FLA...|
|             RED WOOLLY HOTTIE...|
|             SET 7 BABUSHKA NE...|
|             GLASS STAR FROSTE...|
|             HAND WARMER UNION...|
|             HAND WARMER RED P...|
|             ASSORTED COLOUR B...|
|             POPPY'S PLAYHOUSE...|
|             POPPY'S PLAYHOUSE...|
|             FELTCRAFT PRINCES...|
|             IVORY KNITTED MUG...|
|             BOX OF 6 ASSORTED...|
|             BOX OF VINTAGE JI...|
|             BOX OF VINTAGE AL...|
|             HOME BUILDING BLO...|
|             LOVE BUILDING BLO...|
|             RECIPE BOX WITH M...|
+---------------------------------+
only showing top 20 rows



### 6.7.2 ifnull, nullIf, nvl, nvl2
+ SQL 함수이며 DataFrame의 select 표현식으로 사용 가능
    + ifnull(null, 'return_value') # 두 번째 값을, 아니라면 첫 번째 값을 반환 
    + nullif('value', 'value')     # 두 값이 같으면 null
    + nvl(null, 'return_value')    # 두 번째 값을, 아니라면 첫 번째 값을 반환
    + nvl2('not_null', 'return_value', 'else_value') # 두 번째 값을, 아니라면 세번째 값을 반환

In [40]:
spark.sql("""
SELECT
    ifnull(null, 'return_value'),
    nullif('value', 'value'),
    nvl(null, 'return_value'),
    nvl2('not null', 'return_value', 'else_value')
""").show()

+----------------------------+------------------------+-------------------------+----------------------------------------------+
|ifnull(NULL, 'return_value')|nullif('value', 'value')|nvl(NULL, 'return_value')|nvl2('not null', 'return_value', 'else_value')|
+----------------------------+------------------------+-------------------------+----------------------------------------------+
|                return_value|                    null|             return_value|                                  return_value|
+----------------------------+------------------------+-------------------------+----------------------------------------------+



### 6.7.3 drop
+ null 값을 가진 로우를 제거

In [41]:
df.na.drop()
df.na.drop("any").show(1) # 로우 컬럼값 중 하나라도 null이면 제거
df.na.drop("all").show(1) # 로우 컬럼값 모두 null이면 제거

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 1 row

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
+---

In [42]:
df.na.drop("all", subset=("StockCode", "InvoiceNo")).show(1) # 배열 형태의 컬럼을 인수로 전달 가능

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 1 row



### 6.7.4 fill
+ fill: null을 특정한 값으로 채움

In [43]:
""" null을 포함한 DataFrame 행성 """
from pyspark.sql import Row
from pyspark.sql.types import StructField, StructType, StringType, DoubleType

myManualSchema = StructType([
    StructField("string_null", StringType(), True),
    StructField("string2_null", StringType(), True),
    StructField("number_null", DoubleType(), True)
])

myRows = []
myRows.append(Row("Hello", None, float(5))) # string 컬럼에 null 포함
myRows.append(Row(None, "World", None))     # number 컬럼에 null 포함

myDf = spark.createDataFrame(myRows, myManualSchema)
myDf.show()

+-----------+------------+-----------+
|string_null|string2_null|number_null|
+-----------+------------+-----------+
|      Hello|        null|        5.0|
|       null|       World|       null|
+-----------+------------+-----------+



+ fill 함수는 DataType이 동일한 컬럼의 null만 치완
+ 숫자형 또한 치환할 값의 DataType이 동일해야 함

In [44]:
myDf.na.fill("All null valus become this string").show()
myDf.na.fill(5.0).show() 

+--------------------+--------------------+-----------+
|         string_null|        string2_null|number_null|
+--------------------+--------------------+-----------+
|               Hello|All null valus be...|        5.0|
|All null valus be...|               World|       null|
+--------------------+--------------------+-----------+

+-----------+------------+-----------+
|string_null|string2_null|number_null|
+-----------+------------+-----------+
|      Hello|        null|        5.0|
|       null|       World|        5.0|
+-----------+------------+-----------+



In [45]:
""" 딕셔너리 타입을 사용해서 다수의 컬럼에 fill 메서드를 적용 """
fill_cols_vals = {"number_null": 5.0, "string_null": "No Value"}
myDf.na.fill(fill_cols_vals).show()

+-----------+------------+-----------+
|string_null|string2_null|number_null|
+-----------+------------+-----------+
|      Hello|        null|        5.0|
|   No Value|       World|        5.0|
+-----------+------------+-----------+



### 6.7.5 replace

In [46]:
""" 조건에 따라 다른 값으로 대체 """
myDf.na.replace(["Hello"], ["Hello!"], "string_null").show() # null을 지정하는 방법은?

+-----------+------------+-----------+
|string_null|string2_null|number_null|
+-----------+------------+-----------+
|     Hello!|        null|        5.0|
|       null|       World|       null|
+-----------+------------+-----------+



## 6.8 정렬하기
5장(p.145의) asc_nulls_first, desc_nulls_first, asc_nulls_last, desc_nulls_last 참조

## 6.9 복합 데이터 다루기
+ 복합 데이터 타입에는 구조체(struct), 배열(array), 맵(map)이 있음

### 6.9.1 구조체
+ DataFrame 내부의 DataFrame
+ 다수의 컬럼을 괄호로 묶어 생성 가능
+ 복합 데이터 타입은 다른 DataFrame을 조회하는 것과 동일하게 사용할 수 있음. 유일한 차이점은 문법에 점(.)을 사용하거나 getField 메서드를 사용한다는 것
+ (*) 문자로 모든 값을 조회할 수 있음

In [47]:
from pyspark.sql.functions import struct

complexDF = df.select(struct("Description", "InvoiceNo").alias("complex"))
complexDF.createOrReplaceTempView("complexDF")
complexDF.show(5, False)

+---------------------------------------------+
|complex                                      |
+---------------------------------------------+
|[WHITE HANGING HEART T-LIGHT HOLDER, 536365] |
|[WHITE METAL LANTERN, 536365]                |
|[CREAM CUPID HEARTS COAT HANGER, 536365]     |
|[KNITTED UNION FLAG HOT WATER BOTTLE, 536365]|
|[RED WOOLLY HOTTIE WHITE HEART., 536365]     |
+---------------------------------------------+
only showing top 5 rows



In [48]:
complexDF.select("complex.Description", "complex.InvoiceNo") # 모두 동일
complexDF.select(col("complex").getField("Description"), col("complex").getField("InvoiceNo"))
complexDF.select("complex.*")
complexDF.select(col("complex.*"))
complexDF.selectExpr("complex.*").show(5)

+--------------------+---------+
|         Description|InvoiceNo|
+--------------------+---------+
|WHITE HANGING HEA...|   536365|
| WHITE METAL LANTERN|   536365|
|CREAM CUPID HEART...|   536365|
|KNITTED UNION FLA...|   536365|
|RED WOOLLY HOTTIE...|   536365|
+--------------------+---------+
only showing top 5 rows



### 6.9.2 배열
+ 데이터에서 Description 컬럼의 모든 단어를 하나의 로우로 변환

#### Split

+ split 함수에 구분자(delimiter)를 인수로 전달해 배열로 변환

In [49]:
""" 컬럼을 배열로 변환 """
from pyspark.sql.functions import split

df.select(split(col("Description"), " ")).show(2)

+-------------------------+
|split(Description,  , -1)|
+-------------------------+
|     [WHITE, HANGING, ...|
|     [WHITE, METAL, LA...|
+-------------------------+
only showing top 2 rows



In [50]:
""" 배열값의 조회 """
df.select(split(col("Description"), " ").alias("array_col"))\
    .selectExpr("array_col[0]").show(2)

+------------+
|array_col[0]|
+------------+
|       WHITE|
|       WHITE|
+------------+
only showing top 2 rows



#### 배열의 길이

In [51]:
""" size 함수 """
from pyspark.sql.functions import size

df.select(size(split(col("Description"), " "))).show(2)

+-------------------------------+
|size(split(Description,  , -1))|
+-------------------------------+
|                              5|
|                              3|
+-------------------------------+
only showing top 2 rows



#### array_contains
+ array_contains 함수를 사용해 배열에 특정 값이 존재하는지 확인

In [52]:
from pyspark.sql.functions import array_contains

df.select(array_contains(split(col("Description"), " "), "WHITE")).show(2)

+------------------------------------------------+
|array_contains(split(Description,  , -1), WHITE)|
+------------------------------------------------+
|                                            true|
|                                            true|
+------------------------------------------------+
only showing top 2 rows



#### explode
+ 배열 타입의 컬럼을 입력받고 컬럼의 배열값에 포함된 모든 값을 로우로 변환

In [53]:
from pyspark.sql.functions import split, explode
explodedDf = df.withColumn("splitted", split(col("Description"), " "))\
                .withColumn("exploded", explode(col("splitted")))\
                .select("Description", "InvoiceNo", "exploded") # 모든 단어가 하나의 로우로 전환됨

print(df.select("Description").count())
print(explodedDf.select("exploded").count()) # 로우 수가 다름

3108
14414


In [54]:
explodedDf.show(10)

+--------------------+---------+--------+
|         Description|InvoiceNo|exploded|
+--------------------+---------+--------+
|WHITE HANGING HEA...|   536365|   WHITE|
|WHITE HANGING HEA...|   536365| HANGING|
|WHITE HANGING HEA...|   536365|   HEART|
|WHITE HANGING HEA...|   536365| T-LIGHT|
|WHITE HANGING HEA...|   536365|  HOLDER|
| WHITE METAL LANTERN|   536365|   WHITE|
| WHITE METAL LANTERN|   536365|   METAL|
| WHITE METAL LANTERN|   536365| LANTERN|
|CREAM CUPID HEART...|   536365|   CREAM|
|CREAM CUPID HEART...|   536365|   CUPID|
+--------------------+---------+--------+
only showing top 10 rows



In [55]:
explodedDf.select("Description", "exploded").count() # 큰 쪽으로 카운드

14414

In [56]:
explodedDf.select("Description", "exploded").take(10) # Description 컬럼이 Group이 되어 중복됨

[Row(Description='WHITE HANGING HEART T-LIGHT HOLDER', exploded='WHITE'),
 Row(Description='WHITE HANGING HEART T-LIGHT HOLDER', exploded='HANGING'),
 Row(Description='WHITE HANGING HEART T-LIGHT HOLDER', exploded='HEART'),
 Row(Description='WHITE HANGING HEART T-LIGHT HOLDER', exploded='T-LIGHT'),
 Row(Description='WHITE HANGING HEART T-LIGHT HOLDER', exploded='HOLDER'),
 Row(Description='WHITE METAL LANTERN', exploded='WHITE'),
 Row(Description='WHITE METAL LANTERN', exploded='METAL'),
 Row(Description='WHITE METAL LANTERN', exploded='LANTERN'),
 Row(Description='CREAM CUPID HEARTS COAT HANGER', exploded='CREAM'),
 Row(Description='CREAM CUPID HEARTS COAT HANGER', exploded='CUPID')]

### 6.9.3 맵
+ map 함수와 컬럼의 키-값 쌍을 이용해 생성
+ 적합한 키를 사용해 데이터를 조회할 수 있으며, 해당키가 없다면 null값을 반환

In [57]:
""" 맵 생성 """
from pyspark.sql.functions import create_map
df.select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map")).take(2)

[Row(complex_map={'WHITE HANGING HEART T-LIGHT HOLDER': '536365'}),
 Row(complex_map={'WHITE METAL LANTERN': '536365'})]

In [58]:
""" 맵의 데이터 조회 """
df.select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map"))\
    .selectExpr("complex_map['WHITE METAL LANTERN']").show(5)

+--------------------------------+
|complex_map[WHITE METAL LANTERN]|
+--------------------------------+
|                            null|
|                          536365|
|                            null|
|                            null|
|                            null|
+--------------------------------+
only showing top 5 rows



In [59]:
""" 맵의 분해 """
df.select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map"))\
    .selectExpr("explode(complex_map)").show(2)

+--------------------+------+
|                 key| value|
+--------------------+------+
|WHITE HANGING HEA...|536365|
| WHITE METAL LANTERN|536365|
+--------------------+------+
only showing top 2 rows



## 6.10 JSON 다루기

In [60]:
""" Json 컬럼 생성 """
jsonDF = spark.range(1).selectExpr("""
    '{"myJSONKey" : {"myJSONValue" : [1, 2, 3]}}' as jsonString
""")

+ get_json_object 함수로 JSON 객체(딕셔너리나 배열)를 인라인 쿼리로 조회할 수 있음
+ 중첩이 없는 단일 수준의 JSON 객체라면 json_tuple을 사용할 수도 있음

In [61]:
jsonDF.show(10,False)

+-------------------------------------------+
|jsonString                                 |
+-------------------------------------------+
|{"myJSONKey" : {"myJSONValue" : [1, 2, 3]}}|
+-------------------------------------------+



In [62]:
""" 인라인 쿼리로 JSON 조회하기 """
from pyspark.sql.functions import get_json_object, json_tuple

jsonDF.select(
    get_json_object(col("jsonString"), "$.myJSONKey.myJSONValue[1]").alias("column"), # $ 의 의미가 뭐지??
    json_tuple(col("jsonString"), "myJSONKey")
).show(2, False)

+------+-----------------------+
|column|c0                     |
+------+-----------------------+
|2     |{"myJSONValue":[1,2,3]}|
+------+-----------------------+



In [63]:
""" StructType을 Json 문자열로 변경 """
from pyspark.sql.functions import to_json
df.selectExpr("(InvoiceNo, Description) as myStruct")\
    .select(to_json(col("myStruct"))).take(3)

[Row(to_json(myStruct)='{"InvoiceNo":"536365","Description":"WHITE HANGING HEART T-LIGHT HOLDER"}'),
 Row(to_json(myStruct)='{"InvoiceNo":"536365","Description":"WHITE METAL LANTERN"}'),
 Row(to_json(myStruct)='{"InvoiceNo":"536365","Description":"CREAM CUPID HEARTS COAT HANGER"}')]

In [64]:
""" Json 문자열을 객체로 변환 """
from pyspark.sql.functions import from_json
from pyspark.sql.types import *

parseSchema = StructType([
    StructField("InvoiceNo", StringType(), True),
    StructField("Description", StringType(), True)
])

df.selectExpr("(InvoiceNo, Description) as myStruct")\
    .select(to_json(col("myStruct")).alias("newJSON"))\
    .select(from_json(col("newJSON"), parseSchema), col("newJSON")).show(2) # 키를 컬럼명으로 값을 로우로 변경

+--------------------+--------------------+
|  from_json(newJSON)|             newJSON|
+--------------------+--------------------+
|[536365, WHITE HA...|{"InvoiceNo":"536...|
|[536365, WHITE ME...|{"InvoiceNo":"536...|
+--------------------+--------------------+
only showing top 2 rows



## 6.11 사용자 정의 함수 
+ User defined function(UDF)는 레포트별로 데이터를 처리하는 함수이며, SparkSession이나 Context에서 사용할 수 있도록 임시 함수 형태로 등록됨
+ 내장 함수가 제공하는 코드 생성 기능의 장점을 활용할 수 없어 약간의 성능 저하 발생
+ 언어별로 성능차이가 존재, 파이썬에서도 사용할 수 있으므로 자바나 스칼라도 함수 작성을 추천

In [65]:
""" UDF 사용하기 """
udfExDF = spark.range(5).toDF("num")
def power3(double_value):
    return double_value ** 3
power3(2.0)

8.0

In [66]:
""" UDF 등록 및 사용 """
from pyspark.sql.functions import udf
power3udf = udf(power3)

udfExDF.select(power3udf(col("num"))).show(2)

+-----------+
|power3(num)|
+-----------+
|          0|
|          1|
+-----------+
only showing top 2 rows



스칼라에서 등록되 사용자 정의 함수를 파이썬에서 활용:

https://www.cyanny.com/2017/09/15/spark-use-scala-udf-udaf-in-pyspark/

In [67]:
# 사용자 정의 함수를 스파크 SQL로 등록하면 문자열 표현식에서 사용할 수 있게 됨
spark.udf.register("power3py", power3, DoubleType()) # 반환 타입을 Double로 변경하면 null 이 반영됨
udfExDF.selectExpr("power3py(num)").show(2)

+-------------+
|power3py(num)|
+-------------+
|         null|
|         null|
+-------------+
only showing top 2 rows



In [68]:
def power3_f(double_value):
    return float(double_value ** 3)

In [69]:
# 사용자 정의 함수를 스파크 SQL로 등록하면 문자열 표현식에서 사용할 수 있게 됨
spark.udf.register("power3pyf", power3_f, DoubleType()) 
udfExDF.selectExpr("power3pyf(num)").show(2)

+--------------+
|power3pyf(num)|
+--------------+
|           0.0|
|           1.0|
+--------------+
only showing top 2 rows

